

### To Upload the data from mydrive to this notebook



In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [3]:
link1 = 'https://drive.google.com/open?id=18PSgJ2wCNkwr9-DH-oWlsRmIUuCMDMde'
fluff, id_train_unlabled = link1.split('=')
print (id_train_unlabled) # Verify that you have everything after '='


18PSgJ2wCNkwr9-DH-oWlsRmIUuCMDMde


In [0]:
downloaded = drive.CreateFile({'id':id_train_unlabled}) 
downloaded.GetContentFile('train_unlabled.tsv')  

In [5]:
link2 = 'https://drive.google.com/open?id=1KgxQWhq6L8IqcQRuvW-OQIDH6iuq4Lo9'
fluff, id_train_labled = link2.split('=')
print (id_train_labled) # Verify that you have everything after '='


1KgxQWhq6L8IqcQRuvW-OQIDH6iuq4Lo9


In [0]:
downloaded = drive.CreateFile({'id':id_train_labled}) 
downloaded.GetContentFile('train_labled.tsv')

In [7]:
link3 = 'https://drive.google.com/open?id=1z56Wba5iVO9RYuU4NbnHtVVkRW4YfaT8'
fluff, id_test = link3.split('=')
print (id_test) # Verify that you have everything after '='

1z56Wba5iVO9RYuU4NbnHtVVkRW4YfaT8


In [0]:
downloaded = drive.CreateFile({'id':id_test}) 
downloaded.GetContentFile('test.tsv')

In [9]:
link4 = 'https://drive.google.com/open?id=1cTo4HOTu3M3bc-ve7ImPiK4CCbOYnGcO'
fluff, id_sub = link4.split('=')
print (id_sub) # Verify that you have everything after '='


1cTo4HOTu3M3bc-ve7ImPiK4CCbOYnGcO


In [0]:
downloaded = drive.CreateFile({'id':id_sub}) 
downloaded.GetContentFile('sub.csv')

In [11]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-07-17 10:58:58--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-07-17 10:58:58--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-07-17 10:58:59--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

glove

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('glove.6B.zip', 'r')
zip_ref.extractall()
zip_ref.close()


This example applies the HAN classifier to Kaggle's IMDB
review dataset. The goal is to predict whether a review is
positive (5 star rating >=3) or negative (otherwise)

## Importing some required library and functions for data processing



In [13]:
import re
import numpy as np
import pandas as pd
import logging
import sys
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import train_test_split


Using TensorFlow backend.


In [0]:
# Create a logger to provide info on the state of the
# script
stdout = logging.StreamHandler(sys.stdout)
stdout.setFormatter(logging.Formatter(
    '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
))
logger = logging.getLogger('default')
logger.setLevel(logging.INFO)
logger.addHandler(stdout)

In [15]:
#####################################################
# Pre processing                                    #
#####################################################
logger.info("Pre-processsing data.")

# Load Kaggle's IMDB example data
data = pd.read_csv('train_labled.tsv', sep='\t')


2019-07-17 11:01:02,434 - default - INFO - Pre-processsing data.


I0717 11:01:02.434939 140345455339392 <ipython-input-15-404f3efd7689>:1] Pre-processsing data.


In [16]:
data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [0]:
# Do some basic cleaning of the review text
def remove_quotations(text):
    """
    Remove quotations and slashes from the dataset.
    """
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)
    return text

In [0]:
def remove_html(text):
    """
    Very, very raw parser to remove HTML tags from
    texts.
    """
    tags_regex = re.compile(r'<.*?>')
    return tags_regex.sub('', text)

In [0]:
data['review'] = data['review'].apply(remove_quotations)
data['review'] = data['review'].apply(remove_html)
data['review'] = data['review'].apply(lambda x: x.strip().lower())


In [0]:
# Get the data and the sentiment
reviews = data['review'].values
target = data['sentiment'].values

In [21]:
reviews[5:]

array(['i dont know why people think this is such a bad movie. its got a pretty good plot, some good action, and the change of location for harry does not hurt either. sure some of its offensive and gratuitous but this is not the only movie like that. eastwood is in good form as dirty harry, and i liked pat hingle in this movie as the small town cop. if you liked dirty harry, then you should see this one, its a lot better than the dead pool. 4/5',
       'this movie could have been very good, but comes up way short. cheesy special effects and so-so acting. i could have looked past that if the story wasnt so lousy. if there was more of a background story, it would have been better. the plot centers around an evil druid witch who is linked to this woman who gets migraines. the movie drags on and on and never clearly explains anything, it just keeps plodding on. christopher walken has a part, but it is completely senseless, as is most of the movie. this movie had potential, but it looks l

In [22]:
target[5:]

array([1, 0, 0, ..., 0, 0, 1])

In [23]:
len(reviews)

25000

In [24]:
len(target)

25000

In [0]:
MAX_WORDS_PER_SENT = 100
MAX_SENT = 15
MAX_VOC_SIZE = 20000
GLOVE_DIM = 100
TEST_SPLIT = 0.2


In [0]:
del data

In [27]:
#####################################################
# Tokenization                                      #
#####################################################
logger.info("Tokenization.")

# Build a Keras Tokenizer that can encode every token
word_tokenizer = Tokenizer(num_words=MAX_VOC_SIZE)
word_tokenizer.fit_on_texts(reviews)

2019-07-17 11:02:18,967 - default - INFO - Tokenization.


I0717 11:02:18.967431 140345455339392 <ipython-input-27-5333471a0b96>:1] Tokenization.


In [0]:
# Construct the input matrix. This should be a nd-array of
# shape (n_samples, MAX_SENT, MAX_WORDS_PER_SENT).
# We zero-pad this matrix (this does not influence
# any predictions due to the attention mechanism.
X = np.zeros((len(reviews), MAX_SENT, MAX_WORDS_PER_SENT), dtype='int32')

In [0]:
for i, review in enumerate(reviews):
    sentences = sent_tokenize(review)
    tokenized_sentences = word_tokenizer.texts_to_sequences(
        sentences
    )
    tokenized_sentences = pad_sequences(
        tokenized_sentences, maxlen=MAX_WORDS_PER_SENT
    )

    pad_size = MAX_SENT - tokenized_sentences.shape[0]

    if pad_size < 0:
        tokenized_sentences = tokenized_sentences[0:MAX_SENT]
    else:
        tokenized_sentences = np.pad(
            tokenized_sentences, ((0,pad_size),(0,0)),
            mode='constant', constant_values=0
        )

    # Store this observation as the i-th observation in
    # the data matrix
    X[i] = tokenized_sentences[None, ...]

In [0]:
# Transform the labels into a format Keras can handle
y = to_categorical(target)

In [0]:
# We make a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT)


In [32]:
X_train

array([[[    0,     0,     0, ...,  1657,    30,    29],
        [    0,     0,     0, ...,    46,   129,   487],
        [    0,     0,     0, ...,    32,   216,  2992],
        ...,
        [    0,     0,     0, ...,    93,     8,    49],
        [    0,     0,     0, ...,   371,   375,   146],
        [    0,     0,     0, ...,     0,     0,     0]],

       [[    0,     0,     0, ...,  1201,  2795,   619],
        [    0,     0,     0, ...,    28,  2485,  1961],
        [    0,     0,     0, ...,   680,     9,  3805],
        ...,
        [    0,     0,     0, ...,  1127,    14,    38],
        [    0,     0,     0, ...,   900,   784,   239],
        [    0,     0,     0, ...,     0,   300,   159]],

       [[    0,     0,     0, ...,     0,   529,    16],
        [    0,     0,     0, ...,    65,    40,   477],
        [    0,     0,     0, ...,   154,     8,  7102],
        ...,
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,    

## Word Embedding

In [33]:
# Word Embeddings                                   #
#####################################################
logger.info(
    "Creating embedding matrix using pre-trained GloVe vectors."
)

2019-07-17 11:03:21,571 - default - INFO - Creating embedding matrix using pre-trained GloVe vectors.


I0717 11:03:21.571979 140345455339392 <ipython-input-33-cf82a7b6b8bc>:2] Creating embedding matrix using pre-trained GloVe vectors.


In [0]:
# Now, we need to build the embedding matrix. For this we use
# a pretrained (on the wikipedia corpus) 100-dimensional GloVe
# model.

# Load the embeddings from a file
embeddings = {}
with open('glove.6B.%dd.txt' % GLOVE_DIM, encoding='utf-8') as file:
    for line in file:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')

        embeddings[word] = coefs

In [0]:
# Initialize a matrix to hold the word embeddings
embedding_matrix = np.random.random(
    (len(word_tokenizer.word_index) + 1, GLOVE_DIM)
)

# Let the padded indices map to zero-vectors. This will
# prevent the padding from influencing the results
embedding_matrix[0] = 0

In [0]:
# Loop though all the words in the word_index and where possible
# replace the random initalization with the GloVe vector.
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

## Now construct the Layers and Model

### 1. Attention layer

In [0]:
"""containing custom Keras layers that use the attention mechanism."""

import keras
from keras import backend as K



In [0]:
class AttentionLayer(keras.layers.Layer):
    def __init__(self, context_vector_length=100, **kwargs):
        """
        An implementation of a attention layer. This layer
        accepts a 3d Tensor (batch_size, time_steps, input_dim) and
        applies a single layer attention mechanism in the time
        direction (the second axis).
        :param context_vector_lenght: (int) The size of the hidden context vector.
            If set to 1 this layer reduces to a standard attention layer.
        :param kwargs: Any argument that the baseclass Layer accepts.
        """
        self.context_vector_length = context_vector_length
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        dim = input_shape[2]

        # Add a weights layer for the
        self.W = self.add_weight(
            name='W', shape=(dim, self.context_vector_length),
            initializer=keras.initializers.get('uniform'),
            trainable=True
        )

        self.u = self.add_weight(
            name='context_vector', shape=(self.context_vector_length, 1),
            initializer=keras.initializers.get('uniform'),
            trainable=True
        )

        super(AttentionLayer, self).build(input_shape)

    def _get_attention_weights(self, X):
        """
        Computes the attention weights for each timestep in X
        :param X: 3d-tensor (batch_size, time_steps, input_dim)
        :return: 2d-tensor (batch_size, time_steps) of attention weights
        """
        # Compute a time-wise stimulus, i.e. a stimulus for each
        # time step. For this first compute a hidden layer of
        # dimension self.context_vector_length and take the
        # similarity of this layer with self.u as the stimulus
        u_tw = K.tanh(K.dot(X, self.W))
        tw_stimulus = K.dot(u_tw, self.u)

        # Remove the last axis an apply softmax to the stimulus to
        # get a probability.
        tw_stimulus = K.reshape(tw_stimulus, (-1, tw_stimulus.shape[1]))
        att_weights = K.softmax(tw_stimulus)

        return att_weights

    def call(self, X):
        att_weights = self._get_attention_weights(X)

        # Reshape the attention weights to match the dimensions of X
        att_weights = K.reshape(att_weights, (-1, att_weights.shape[1], 1))
        att_weights = K.repeat_elements(att_weights, X.shape[-1], -1)

        # Multiply each input by its attention weights
        weighted_input = keras.layers.Multiply()([X, att_weights])

        # Sum in the direction of the time-axis.
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

    def get_config(self):
        config = {
            'context_vector_length': self.context_vector_length
        }
        base_config = super(AttentionLayer, self).get_config()
        return {**base_config, **config}

### 2. Model preperation 

In [0]:
import keras
from keras.layers import (
    Dense, GRU, TimeDistributed, Input,
    Embedding, Bidirectional, Lambda
)
from keras.models import Model
# from keras_han.layers import AttentionLayer



In [0]:
class HAN(Model):
    def __init__(
            self, max_words, max_sentences, output_size,
            embedding_matrix, word_encoding_dim=200,
            sentence_encoding_dim=200, inputs=None,
            outputs=None, name='han-for-docla'
    ):
        """
        A Keras implementation of Hierarchical Attention networks
        for document classification.
        :param max_words: The maximum number of words per sentence
        :param max_sentences: The maximum number of sentences
        :param output_size: The dimension of the last layer (i.e.
            the number of classes you wish to predict)
        :param embedding_matrix: The embedding matrix to use for
            representing words
        :param word_encoding_dim: The dimension of the GRU
            layer in the word encoder.
        :param sentence_encoding_dim: The dimension of the GRU
            layer in the sentence encoder.
        """
        self.max_words = max_words
        self.max_sentences = max_sentences
        self.output_size = output_size
        self.embedding_matrix = embedding_matrix
        self.word_encoding_dim = word_encoding_dim
        self.sentence_encoding_dim = sentence_encoding_dim


        in_tensor, out_tensor = self._build_network()

        super(HAN, self).__init__(
            inputs=in_tensor, outputs=out_tensor, name=name
        )

    def build_word_encoder(self, max_words, embedding_matrix, encoding_dim=200):
        """
        Build the model that embeds and encodes in context the
        words used in a sentence. The return model takes a tensor of shape
        (batch_size, max_length) that represents a collection of sentences
        and returns an encoded representation of these sentences.
        :param max_words: (int) The maximum sentence length this model accepts
        :param embedding_matrix: (2d array-like) A matrix with the i-th row
            representing the embedding of the word represented by index i.
        :param encoding_dim: (int, should be even) The dimension of the
            bidirectional encoding layer. Half of the nodes are used in the
            forward direction and half in the backward direction.
        :return: Instance of keras.Model
        """
        assert encoding_dim % 2 == 0, "Embedding dimension should be even"

        vocabulary_size = embedding_matrix.shape[0]
        embedding_dim = embedding_matrix.shape[1]

        embedding_layer = Embedding(
            vocabulary_size, embedding_dim,
            weights=[embedding_matrix], input_length=max_words,
            trainable=False
        )

        sentence_input = Input(shape=(max_words,), dtype='int32')
        embedded_sentences = embedding_layer(sentence_input)
        encoded_sentences = Bidirectional(
            GRU(int(encoding_dim / 2), return_sequences=True)
        )(embedded_sentences)

        return Model(
            inputs=[sentence_input], outputs=[encoded_sentences], name='word_encoder'
        )

    def build_sentence_encoder(self, max_sentences, summary_dim, encoding_dim=200):
        """
        Build the encoder that encodes the vector representation of
        sentences in their context.
        :param max_sentences: The maximum number of sentences that can be
            passed. Use zero-padding to supply shorter sentences.
        :param summary_dim: (int) The dimension of the vectors that summarizes
            sentences. Should be equal to the encoding_dim of the word
            encoder.
        :param encoding_dim: (int, even) The dimension of the vector that
            summarizes sentences in context. Half is used in forward direction,
            half in backward direction.
        :return: Instance of keras.Model
        """
        assert encoding_dim % 2 == 0, "Embedding dimension should be even"

        text_input = Input(shape=(max_sentences, summary_dim))
        encoded_sentences = Bidirectional(
            GRU(int(encoding_dim / 2), return_sequences=True)
        )(text_input)
        return Model(
            inputs=[text_input], outputs=[encoded_sentences], name='sentence_encoder'
        )

    def _build_network(self):
        """
        Build the graph that represents this network
        :return: in_tensor, out_tensor, Tensors representing the input and output
            of this network.
        """
        in_tensor = Input(shape=(self.max_sentences, self.max_words))

        word_encoder = self.build_word_encoder(
            self.max_words, self.embedding_matrix, self.word_encoding_dim
        )

        word_rep = TimeDistributed(
            word_encoder, name='word_encoder'
        )(in_tensor)

        # Sentence Rep is a 3d-tensor (batch_size, max_sentences, word_encoding_dim)
        sentence_rep = TimeDistributed(
            AttentionLayer(), name='word_attention'
        )(word_rep)

        doc_rep = self.build_sentence_encoder(
            self.max_sentences, self.word_encoding_dim, self.sentence_encoding_dim
        )(sentence_rep)

        # We get the final representation by applying our attention mechanism
        # to the encoded sentences
        doc_summary = AttentionLayer(name='sentence_attention')(doc_rep)

        out_tensor = Dense(
            self.output_size, activation='softmax', name='class_prediction'
        )(doc_summary)

        return in_tensor, out_tensor

    def get_config(self):
        config = {
            'max_words': self.max_words,
            'max_sentences': self.max_sentences,
            'output_size': self.output_size,
            'embedding_matrix': self.embedding_matrix,
            'word_encoding_dim': self.word_encoding_dim,
            'sentence_encoding_dim': self.sentence_encoding_dim,
            'base_config': super(HAN, self).get_config()
        }

        return config

    @classmethod
    def from_config(cls, config, custom_objects=None):
        """
        Keras' API isn't really extendible at this point
        therefore we need to use a bit hacky solution to
        be able to correctly reconstruct the HAN model
        from a config. This therefore does not reconstruct
        a instance of HAN model, but actually a standard
        Keras model that behaves exactly the same.
        """
        base_config = config.pop('base_config')

        return Model.from_config(
            base_config, custom_objects=custom_objects
        )

    def predict_sentence_attention(self, X):
        """
        For a given set of texts predict the attention
        weights for each sentence.
        :param X: 3d-tensor, similar to the input for predict
        :return: 2d array (num_obs, max_sentences) containing
            the attention weights for each sentence
        """
        att_layer = self.get_layer('sentence_attention')
        prev_tensor = att_layer.input

        # Create a temporary dummy layer to hold the
        # attention weights tensor
        dummy_layer = Lambda(
            lambda x: att_layer._get_attention_weights(x)
        )(prev_tensor)

        return Model(self.input, dummy_layer).predict(X)

## Model Training

In [41]:
logger.info("Training the model.")


han_model = HAN(
    MAX_WORDS_PER_SENT, MAX_SENT, 2, embedding_matrix,
    word_encoding_dim=100, sentence_encoding_dim=100
)

han_model.summary()

han_model.compile(
    optimizer='adagrad', loss='categorical_crossentropy',
    metrics=['acc']
)



2019-07-17 11:04:30,743 - default - INFO - Training the model.


I0717 11:04:30.743460 140345455339392 <ipython-input-41-01420d199258>:1] Training the model.
W0717 11:04:30.754431 140345455339392 module_wrapper.py:136] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 11:04:32.658465 140345455339392 module_wrapper.py:136] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/util/module_wrapper.py:163: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
word_encoder (TimeDistribute (None, 15, 100, 100)      8195700   
_________________________________________________________________
word_attention (TimeDistribu (None, 15, 100)           10100     
_________________________________________________________________
sentence_encoder (Model)     (None, 15, 100)           45300     
_________________________________________________________________
sentence_attention (Attentio (None, 100)               10100     
_________________________________________________________________
class_prediction (Dense)     (None, 2)                 202       
Total params: 8,261,402
Trainable params: 111,002
Non-trainable params: 8,150,400
____________________________________________________________

In [45]:



han_model.fit(
    X_train, y_train, batch_size=20, epochs=10,
    validation_data=(X_test, y_test)
)

W0717 11:10:31.769526 140345455339392 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/math_grad.py:1251: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 842s 42ms/step - loss: 0.4124 - acc: 0.8028 - val_loss: 0.3400 - val_acc: 0.8548
Epoch 2/10
20000/20000 [==============================] - 838s 42ms/step - loss: 0.3162 - acc: 0.8658 - val_loss: 0.3179 - val_acc: 0.8636
Epoch 3/10
20000/20000 [==============================] - 838s 42ms/step - loss: 0.2958 - acc: 0.8753 - val_loss: 0.3118 - val_acc: 0.8662
Epoch 4/10
20000/20000 [==============================] - 836s 42ms/step - loss: 0.2794 - acc: 0.8839 - val_loss: 0.3058 - val_acc: 0.8648
Epoch 5/10
20000/20000 [==============================] - 841s 42ms/step - loss: 0.2689 - acc: 0.8883 - val_loss: 0.2999 - val_acc: 0.8714
Epoch 6/10
20000/20000 [==============================] - 844s 42ms/step - loss: 0.2580 - acc: 0.8937 - val_loss: 0.2994 - val_acc: 0.8682
Epoch 7/10
20000/20000 [==============================] - 842s 42ms/step - loss: 0.2495 - acc: 0.8992 - val_loss: 0.3